# Exp6: 基于集成学习的 Amazon 用户评论质量预测

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


读取数据

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
#读取train数据
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/xtzx/hw6/data/train.csv', sep='\t')
train_df                    

,reviewerID,asin,reviewText,overall,votes_up,votes_all,label
0,7885,3901,"First off, allow me to correct a common mistak...",5.0,6,7,0
1,52087,47978,I am really troubled by this Story and Enterta...,3.0,99,134,0
2,5701,3667,A near-perfect film version of a downright glo...,4.0,14,14,1
3,47191,40892,Keep your expectations low. Really really low...,1.0,4,7,0
4,40957,15367,"""they dont make em like this no more...""well.....",5.0,3,6,0
...,...,...,...,...,...,...,...
57034,58315,29374,"If you like beautifully shot, well acted films...",2.0,12,21,0
57035,23328,45548,This is a great set of films Wayne did Fox and...,5.0,15,18,0
57036,27203,42453,It's what's known as a comedy of manners. It's...,3.0,4,5,0
57037,33992,44891,Ellen can do no wrong as far a creating wonder...,5.0,4,5,0


In [ ]:
#读取test数据
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/xtzx/hw6/data/test.csv', sep='\t')
test_df

,Id,reviewerID,asin,reviewText,overall
0,0,82947,37386,I REALLY wanted this series but I am in SHOCK ...,1.0
1,1,10154,23543,I have to say that this is a work of art for m...,4.0
2,2,5789,5724,Alien 3 is certainly the most controversal fil...,3.0
3,3,9198,5909,"I love this film...preachy? Well, of course i...",5.0
4,4,33252,21214,Even though I previously bought the Gamera Dou...,5.0
...,...,...,...,...,...
11203,11203,18250,35309,I honestly never heard of the graphic novel un...,5.0
11204,11204,3200,2130,Archie Bunker's command to stifle YOURSELF! wa...,5.0
11205,11205,37366,41971,"In LSD - My Problem Child, Albert Hoffman wrot...",5.0
11206,11206,1781,33089,I have owned this DVD for over a year now and ...,5.0


清理文本

In [ ]:
#调用nltk库，删除文本中的“stopword”, 并进行词干化（stemming）/词元化（lemmatisation），最后，只取前20个词返回。储存在‘text_clean’列中。

import nltk ## for language detection 
import re
''' 
Preprocess a string. 
:parameter 
    :param text: string - name of column containing text 
    :param lst_stopwords: list - list of stopwords to remove 
    :param flg_stemm: bool - whether stemming is to be applied 
    :param flg_lemm: bool - whether lemmitisation is to be applied 
    :param max_len: int - max number of words in the text
:return 
    cleaned text 
''' 
nltk.download('stopwords')
nltk.download('wordnet')
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None, max_len = 20): 
    ## clean (convert to lowercase and remove punctuations and characters and then strip) 
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip()) 
 
    ## Tokenize (convert from string to list) 
    lst_text = text.split()    ## remove Stopwords 
    if lst_stopwords is not None: 
        lst_text = [word for word in lst_text if word not in  
                    lst_stopwords] 
 
    ## Stemming (remove -ing, -ly, ...) 
    if flg_stemm == True: 
        ps = nltk.stem.porter.PorterStemmer() 
        lst_text = [ps.stem(word) for word in lst_text] 
 
    ## Lemmatisation (convert the word into root word) 
    if flg_lemm == True: 
        lem = nltk.stem.wordnet.WordNetLemmatizer() 
        lst_text = [lem.lemmatize(word) for word in lst_text] 
      
    lst_text = lst_text[:max_len]
 
    ## back to string from list 
    text = " ".join(lst_text) 
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from nltk.corpus import stopwords

lst_stopwords = nltk.corpus.stopwords.words("english") 
maxlen = 20
train_df["text_clean"] = train_df["reviewText"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,
                                                                                      lst_stopwords=lst_stopwords, max_len = maxlen)) 
clean_df = train_df.copy()

In [ ]:
# 新增情绪列
from textblob import TextBlob
clean_df["sentiment"] = clean_df['reviewText'].apply(lambda x: TextBlob(x).sentiment.polarity)

#增加计数列
clean_df['word_count'] = clean_df["reviewText"].apply(lambda x: len(str(x).split(" "))) 
clean_df['char_count'] = clean_df["reviewText"].apply(lambda x: sum(len(word) for word in str(x).split(" "))) 
clean_df['sentence_count'] = clean_df["reviewText"].apply(lambda x: len(str(x).split("."))) 
clean_df['avg_word_length'] = clean_df['char_count'] / clean_df['word_count'] 
clean_df['avg_sentence_lenght'] = clean_df['word_count'] / clean_df['sentence_count'] 

# clean_df = clean_df.drop(columns = ['reviewText'])
clean_df.head()

,reviewerID,asin,reviewText,overall,votes_up,votes_all,label,text_clean,sentiment,word_count,char_count,sentence_count,avg_word_length,avg_sentence_lenght
0,7885,3901,"First off, allow me to correct a common mistak...",5.0,6,7,0,first allow correct common mistake made one re...,0.198494,416,1987,13,4.776442,32.000000
1,52087,47978,I am really troubled by this Story and Enterta...,3.0,99,134,0,really troubled story entertained way presente...,0.030000,477,2225,21,4.664570,22.714286
2,5701,3667,A near-perfect film version of a downright glo...,4.0,14,14,1,nearperfect film version downright gloomy play...,0.221667,71,357,4,5.028169,17.750000
3,47191,40892,Keep your expectations low. Really really low...,1.0,4,7,0,keep expectation low really really low may ent...,-0.106471,117,577,14,4.931624,8.357143
4,40957,15367,"""they dont make em like this no more...""well.....",5.0,3,6,0,dont make em like morewell throw phrase window...,0.037500,98,461,21,4.704082,4.666667


In [ ]:
# clean_df.to_csv('/content/drive/MyDrive/Colab Notebooks/xtzx/hw6/data/clean.csv', sep = '\t')

In [ ]:
lst_stopwords = nltk.corpus.stopwords.words("english") 
maxlen = 20
test_df["text_clean"] = test_df["reviewText"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,
                                                                                      lst_stopwords=lst_stopwords, max_len = maxlen)) 

# 新增情绪列
from textblob import TextBlob
test_df["sentiment"] = test_df['reviewText'].apply(lambda x: TextBlob(x).sentiment.polarity)

#增加计数列
test_df['word_count'] = test_df["reviewText"].apply(lambda x: len(str(x).split(" "))) 
test_df['char_count'] = test_df["reviewText"].apply(lambda x: sum(len(word) for word in str(x).split(" "))) 
test_df['sentence_count'] = test_df["reviewText"].apply(lambda x: len(str(x).split("."))) 
test_df['avg_word_length'] = test_df['char_count'] / test_df['word_count'] 
test_df['avg_sentence_lenght'] = test_df['word_count'] / test_df['sentence_count'] 

test_df.head()


,Id,reviewerID,asin,reviewText,overall,text_clean,sentiment,word_count,char_count,sentence_count,avg_word_length,avg_sentence_lenght
0,0,82947,37386,I REALLY wanted this series but I am in SHOCK ...,1.0,really wanted series shock price series would ...,0.200000,39,158,4,4.051282,9.750000
1,1,10154,23543,I have to say that this is a work of art for m...,4.0,say work art movie production value definitely...,0.296491,169,777,11,4.597633,15.363636
2,2,5789,5724,Alien 3 is certainly the most controversal fil...,3.0,alien 3 certainly controversal film alien seri...,0.016857,365,1699,21,4.654795,17.380952
3,3,9198,5909,"I love this film...preachy? Well, of course i...",5.0,love filmpreachy well course also lyrical beau...,0.333766,71,326,14,4.591549,5.071429
4,4,33252,21214,Even though I previously bought the Gamera Dou...,5.0,even though previously bought gamera double fe...,0.246667,78,384,6,4.923077,13.000000


In [ ]:
test_df.to_csv('/content/drive/MyDrive/Colab Notebooks/xtzx/hw6/data/clean_test.csv', sep = '\t')

bert特征提取

In [ ]:
#bert embedding 提取前20个有效词中的文本特种，返回值为768维向量

import time
import torch
import transformers as ppb
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') #加载预训练模型
model1 = BertModel.from_pretrained("bert-base-uncased")
model = model1.to('cuda') # If you have GPU shift the model to GPU #利用GPU进行加速



In [ ]:
#提取测试集中第一个数据的文本特征
text = test_df['text_clean'][0] #取出文本
encoded_input = tokenizer(text, return_tensors='pt').to('cuda') #将数据传到GPU
output = model(**encoded_input)
feature = output[0][:,0,:].cpu().detach().numpy() #取出需要的特征，768维向量
feature.shape

(1, 768)

In [ ]:
#提取测试集中第二个数据的文本特征
text = test_df['text_clean'][1]
encoded_input = tokenizer(text, return_tensors='pt').to('cuda')
output = model(**encoded_input)
feature2 = output[0][:,0,:].cpu().detach().numpy()
feature2.shape

(1, 768)

In [ ]:
test_bert = np.vstack((feature, feature2))
test_bert

array([[-0.16238534, -0.5062264 ,  0.34817144, ..., -0.61204606,
         0.24835233,  0.0076165 ],
       [-0.37093663, -0.17470673, -0.09768231, ..., -0.6112864 ,
         0.40850037,  0.09287984]], dtype=float32)

In [ ]:
t0 = time()
for i in range(2, 11208):
  text = test_df['text_clean'][i]
  encoded_input = tokenizer(text, return_tensors='pt').to('cuda')
  output = model(**encoded_input)
  featurei = output[0][:,0,:].cpu().detach().numpy()
  test_bert = np.vstack((test_bert, featurei))
  if i % 1000 == 0:
    print(i)

print("done in ", time() - t0, "sec")

In [ ]:
test_bert.shape

(11208, 768)

In [ ]:
#将提取好的特征保存成numpy矩阵格式，方便下次调用
# np.save('/content/drive/MyDrive/Colab Notebooks/xtzx/hw6/data/bert_clean_test.npy',test_bert) 

In [ ]:
del test_bert #释放内存

In [ ]:
#提取train集中第一个文本的特征
text = train_df['text_clean'][0]
encoded_input = tokenizer(text, return_tensors='pt').to('cuda')
output = model(**encoded_input)

In [ ]:
feature = output[0][:,0,:].cpu().detach().numpy()
feature.shape

(1, 768)

In [ ]:
#提取train集中第二个文本的特征
text = train_df['text_clean'][1]
encoded_input = tokenizer(text, return_tensors='pt').to('cuda')
output = model(**encoded_input)
feature2 = output[0][:,0,:].cpu().detach().numpy()
feature2.shape

(1, 768)

In [ ]:
old = np.vstack((feature, feature2))
all_bert = old.copy()

In [ ]:
all_bert

array([[-0.4800572 , -0.03585712,  0.23369914, ..., -0.4476912 ,
         0.51827276,  0.46136004],
       [-0.37914237,  0.09411145,  0.05395877, ..., -0.47702298,
         0.60461056,  0.29169947]], dtype=float32)

In [ ]:
#提取train集中所有文本的特征
t0 = time()
for i in range(2, 57039):
  text = train_df['text_clean'][i]
  encoded_input = tokenizer(text, return_tensors='pt').to('cuda')
  output = model(**encoded_input)
  featurei = output[0][:,0,:].cpu().detach().numpy()
  all_bert = np.vstack((all_bert, featurei))
  if i % 1000 == 0:
    print(i)
    np.save('/content/drive/MyDrive/Colab Notebooks/xtzx/hw6/data/bert_clean.npy', all_bert)

print("done in ", time() - t0, "sec")

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
done in  2243.788642883301 sec


In [ ]:
#保存train集的文本特征
# np.save('/content/drive/MyDrive/Colab Notebooks/xtzx/hw6/data/bert_clean_train.npy', all_bert)

In [ ]:
del all_bert #释放内存

看到这里或许就可以停了，后面是其他的一些尝试（有的成功，有的失败），但由于运行速度太慢或者占内存太多无法顺利运行等原因最终没有使用。

In [ ]:
#把content的内容缩减到200个字符（约40词） 之后再加
# maxchar = 200
# train_df['content'] = 0
# for i in range(len(train_df['reviewText'])):
#   train_df['content'][i] = train_df['reviewText'][i][:maxchar]

# test_df['content'] = 0
# for i in range(len(test_df['reviewText'])):
#   test_df['content'][i] = test_df['reviewText'][i][:maxchar]

#把内容缩减到20个词
# maxlen = 20
# train_df['fcontent'] = 0
# for i in range(len(train_df['reviewText'])):
#     train_df['fcontent'][i] = " ".join(train_df['reviewText'][i].split(sep = ' ')[:maxlen])

In [ ]:
# test_df['fcontent'] = 0
# for i in range(len(test_df['reviewText'])):
#     test_df['fcontent'][i] = " ".join(test_df['reviewText'][i].split(sep = ' ')[:maxlen])

# test_df

,Id,reviewerID,asin,reviewText,overall,fcontent
0,0,82947,37386,I REALLY wanted this series but I am in SHOCK ...,1.0,I REALLY wanted this series but I am in SHOCK ...
1,1,10154,23543,I have to say that this is a work of art for m...,4.0,I have to say that this is a work of art for m...
2,2,5789,5724,Alien 3 is certainly the most controversal fil...,3.0,Alien 3 is certainly the most controversal fil...
3,3,9198,5909,"I love this film...preachy? Well, of course i...",5.0,"I love this film...preachy? Well, of course i..."
4,4,33252,21214,Even though I previously bought the Gamera Dou...,5.0,Even though I previously bought the Gamera Dou...
...,...,...,...,...,...,...
11203,11203,18250,35309,I honestly never heard of the graphic novel un...,5.0,I honestly never heard of the graphic novel un...
11204,11204,3200,2130,Archie Bunker's command to stifle YOURSELF! wa...,5.0,Archie Bunker's command to stifle YOURSELF! wa...
11205,11205,37366,41971,"In LSD - My Problem Child, Albert Hoffman wrot...",5.0,"In LSD - My Problem Child, Albert Hoffman wrot..."
11206,11206,1781,33089,I have owned this DVD for over a year now and ...,5.0,I have owned this DVD for over a year now and ...


In [ ]:
#小训练集选取时，label 1 和 0 的数量相同
small_batch_df = pd.concat([train_df[train_df['label'] == 0][:12000], train_df[train_df['label'] == 1][:12000]])
small_batch_df
from sklearn.utils import shuffle
small_batch_df = shuffle(small_batch_df, random_state = 520)

small_batch_df

,content,label
2936,Maxium Overdrive written and directed by Steph...,0
10566,Thank you New Video Group for bringing us this...,0
5667,Masterful work of Italian neo-realism by the g...,0
10074,For young and old who are fascinated with dino...,1
8621,I saw this movie on the satellite and went and...,0
...,...,...
5266,"I have seen many a cool collector's movie set,...",0
39874,I can't tell you how long I waited in trepidat...,1
21290,"This is a well-written, well acted, very origi...",1
4812,This is the same wonderful series I remember f...,0


In [ ]:
len(train_df['content'][100])

115

In [ ]:
test_df['content'][1]

'I have to say that this is a work of art for movies. The production values are definitely some of'

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.5MB 5.9MB/s 
     |████████████████████████████████| 890kB 38.8MB/s 
     |████████████████████████████████| 2.9MB 41.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=021b959a5da1620d3eaf033061330836197b2edfc22a06f662ac2b4d785119ef
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
whole_X = pd.concat([small_batch_df, test_df], keys = ['train', 'test'])

In [ ]:
whole_X

content  label       Id
train 2936   Maxium Overdrive written and directed by Steph...    0.0      NaN
      10566  Thank you New Video Group for bringing us this...    0.0      NaN
      5667   Masterful work of Italian neo-realism by the g...    0.0      NaN
      10074  For young and old who are fascinated with dino...    1.0      NaN
      8621   I saw this movie on the satellite and went and...    0.0      NaN
...                                                        ...    ...      ...
test  11203  I honestly never heard of the graphic novel un...    NaN  11203.0
      11204  Archie Bunker's command to stifle YOURSELF! wa...    NaN  11204.0
      11205  In LSD - My Problem Child, Albert Hoffman wrot...    NaN  11205.0
      11206  I have owned this DVD for over a year now and ...    NaN  11206.0
      11207  This movie is just a slap in the face [or othe...    NaN  11207.0

[35208 rows x 3 columns]

In [ ]:
whole_X['label'].value_counts()

1.0    12000
0.0    12000
Name: label, dtype: int64

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
tokenizer

PreTrainedTokenizer(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
tokenized = whole_X['content'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
padded[1]

array([  101,  4067,  2017,  2047,  2678,  2177,  2005,  5026,  2149,
        2023, 13164,  2275,  1997,  7742, 11721, 24291,  1998,  2074,
        2066,  1996, 16794, 13164,   102,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0])

In [ ]:
np.array(padded).shape

(35208, 111)

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(35208, 111)

In [ ]:
labels = whole_X['label']
labels.value_counts()

1.0    12000
0.0    12000
Name: label, dtype: int64

In [ ]:
#测试用
padded[1]
np.array(padded[1]).reshape(1, -1)

array([[ 101, 5860, 1015, 1010, 2217, 1037, 1024, 3873, 4665, 1006, 2727,
        1011, 2722, 1007, 1025, 2217, 1038, 1024, 2709, 2000, 3873, 4665,
        1006, 2722, 1011, 2687, 1007, 5860, 1016, 1010, 2217, 1037, 1024,
        3873, 4665,  102,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0]])

In [ ]:
#直接用bert模型一个个提取特征
# for j in range(1,352):
#   print(j)
f100b = []
#   i0 = j * 100
#   i1 = i0 + 100
for i in range(35200, 35208):
  pad1 = np.array(padded[i]).reshape(1, -1)
  am1 = np.array(attention_mask[i]).reshape(1, -1)

  input_ids = torch.tensor(pad1)  
  attention_mask2 = torch.tensor(am1)

  with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask2)
  
  features = last_hidden_states[0][:,0,:].numpy()

  f100b.append(features)

f100b = np.array(f100b)
f100b = f100b.reshape(8, -1)

tryyy = np.load('/content/drive/MyDrive/Colab Notebooks/xtzx/hw6/data/bertf3.npy')

save_array_new = np.hstack((f100b, np.array(whole_X['label'][35200:35208]).reshape(8, -1)))
save_array = np.vstack((tryyy, save_array_new))
# np.save('/content/drive/MyDrive/Colab Notebooks/xtzx/hw6/data/bertf3.npy', save_array)


In [ ]:
f100b = np.array(f100b)
f100b = f100b.reshape(100, -1)

In [ ]:
save_array_new = np.hstack((f100b, np.array(whole_X['label'][100:200]).reshape(100, -1)))
save_array = np.vstack((tryy, save_array_new))
np.save('/content/drive/MyDrive/Colab Notebooks/xtzx/hw6/data/bertf2.npy', save_array)

In [ ]:
np.array(whole_X['label'][:100])

array([1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1.,
       0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1.,
       1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1.,
       0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
save_array = np.hstack((f100a, np.array(whole_X['label'][:100]).reshape(100, -1)))

np.save('/content/drive/MyDrive/Colab Notebooks/xtzx/hw6/data/bertf.npy', save_array)

In [ ]:
tryy = np.load('/content/drive/MyDrive/Colab Notebooks/xtzx/hw6/data/bertf.npy')
np.save('/content/drive/MyDrive/Colab Notebooks/xtzx/hw6/data/bertf3.npy', tryy)

In [ ]:
tryyy = np.load('/content/drive/MyDrive/Colab Notebooks/xtzx/hw6/data/bertf3.npy')
tryyy.shape

(35200, 769)

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(f100a, labels, train_size = 80, shuffle = False)

lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

print("train score:", lr_clf.score(train_features, train_labels))

pred = lr_clf.predict(test_features)


train score: 0.9875


In [ ]:
print("test score:", lr_clf.score(test_features, test_labels))

test score: 0.65


(100, 768)

In [ ]:
def bertpred(padded, attention_mask, labels):
  ti = time()
  input_ids = torch.tensor(padded)  
  attention_mask = torch.tensor(attention_mask)

  with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
  
  features = last_hidden_states[0][:,0,:].numpy()

  train_features, test_features, train_labels, test_labels = train_test_split(features, labels, train_size = 1000, shuffle = False)

  lr_clf = LogisticRegression()
  lr_clf.fit(train_features, train_labels)

  print("train score:", lr_clf.score(train_features, train_labels))

  pred = lr_clf.predict(test_features)

  print("pred done in", ti - time(), 'sec')

  return pred



In [ ]:
# Ada+bert
# padded / attention_mask 前24000是train，后11208是test
# 想要反复利用前面24000，推测后面11208的标签
train_padded = padded[:24000]
test_padded = padded[24000:]
train_am = attention_mask[:24000]
test_am = attention_mask[24000:]
trlabels = whole_X['label'][:24000]

In [ ]:
train_p.shape, train_a.shape, train_labels.shape, notuse_p.shape, notuse_a.shape, notuse_l.shape

NameError: ignored

In [ ]:
np.array(whole_X['label'][24000:])

array([nan, nan, nan, ..., nan, nan, nan])

In [ ]:
#先对前1000个数据进行预测
from time import time
test_am1000 = test_am[:1000]
test_padded1000 = test_padded[:1000]
pred1000 = []
for i in range(5):
  i = 0
  ti = time()
  train_p, notuse_p, train_a,  notuse_a, train_labels, notuse_l = train_test_split(train_padded, train_am, trlabels, train_size = 1000, random_state=520 + i)
  newpad = np.vstack((train_p, test_padded1000))
  newam = np.vstack((train_a, test_am1000))
  newlabels = np.hstack((np.array(train_labels), np.array(whole_X['label'][24000:25000])))

  pred = bertpred(padded, attention_mask, newlabels)

  pred1000.append(pred)

  print("round ", i , "done in ", ti - time(), "sec")



In [ ]:
#2000个数据测试一下

tokenized = whole_X['content'][:2000].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape


small_padded = padded[:2000]
small_am = attention_mask[:2000]
small_labels = whole_X['label'][:2000]

small_padded.shape, small_am.shape, small_labels.shape

((2000, 62), (2000, 62), (2000,))

In [ ]:

from time import time
t0 = time()
input_ids = torch.tensor(small_padded)  
am_tensor = torch.tensor(small_am)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=am_tensor)

print("done in ", time() - t0, 'sec')

features = last_hidden_states[0][:,0,:].numpy()
features.shape()

done in  177.38574266433716 sec


TypeError: ignored

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, small_labels, train_size = 1000, shuffle = False)

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
lr_clf.score(test_features, test_labels)

0.623

In [ ]:
lr_clf.score(train_features, train_labels)

0.845

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, train_size = 1000, shuffle = False)

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
lr_clf.score(test_features, test_labels)

0.748

In [ ]:
lr_clf.score(train_features, train_labels)

0.812

In [ ]:
pred = lr_clf.predict(test_features)

In [ ]:
pred

array([1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1.,
       1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0.,
       1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0.,
       0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1.,
       1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.,
       1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1.,
       0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
       1., 0., 1., 1., 1.

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.631 (+/- 0.06)


In [ ]:
pred = lr_clf.predict(test_features)

In [ ]:
pred

array([1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,